<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7f4ea606f7e05cc3722ac1b07fc3bfc9a72767fa56a05a69b06f33aded2161f5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-30 16:24:38
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.42 C
-------------------
Today PnL: -60.10 K (-0.42%)
Current PnL: -27.24 L (-17.37%)
CY Booked + Current PnL: -12.95 L (-8.26%)
-------------------
Total profit:  1.36 L
Total loss:  -28.60 L
-------------------
Total Booked + Current PnL: 14.03 L (10.8%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.08%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 92.62 L (65.32%)
Deployed:  1.30 C
Current:  1.42 C
CAGR/XIRR %: 4.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,-1.08,-20.39,25.72,0.09,20635.0,-20546.0,80231.0,86.13,34.0,M-SC,3.05,253.0,-1.00,0.57,5.41,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,-0.46,12.32,8.08,21.40,24090.0,32713.0,298139.0,4.17,61.0,X-LC,5.29,12.0,1.36,2.10,19.43,XY25,NTT,FMCG
51,MASFIN,398.61,0.61,-3.72,26.76,22.05,25244.0,-3645.0,94335.0,-16.89,49.0,H-SC,6.52,164.0,-0.14,0.67,37.12,XR,ATH,FINANCE
84,VOLTAS,1530.00,-2.13,5.67,13.27,19.69,26886.0,10863.0,202605.0,-4.67,44.0,X-MC,2.27,78.0,0.40,1.43,12.85,XY25,NTT,AC
44,ITC,452.00,-0.50,-1.82,12.83,10.78,30273.0,-4365.0,235953.0,-41.56,43.0,X-LC,2.02,5.0,-0.14,1.66,3.32,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,17.13,-4.63,54.08,46.95,64625.0,-5799.0,119498.0,-11.74,63.0,H-SC,9.73,154.0,-0.09,0.84,49.70,MH,ATH,POWER
62,REPCOHOME,880.00,2.73,-10.86,112.95,89.82,288461.0,-31117.0,255388.0,-55.29,56.0,H-SC,5.36,160.0,-0.11,1.80,35.12,XY24,NTT,FINANCE
39,INDIAMART,4810.62,2.16,-3.45,114.10,106.72,135877.0,-4250.0,119086.0,-52.15,45.0,H-SC,5.31,139.0,-0.03,0.84,19.13,AR,ATH,MISC
70,STARHEALTH,761.00,1.60,-13.16,69.58,47.26,169859.0,-37003.0,244120.0,18.56,35.0,H-SC,9.71,171.0,-0.22,1.72,30.79,XY24,NTT,INSURANCE
21,DABUR,735.00,1.30,-3.10,48.29,43.70,116084.0,-7678.0,240390.0,-11.07,44.0,X-MC,1.98,72.0,-0.07,1.70,11.49,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJESHEXPO,518.00,-5.00,-62.90,169.72,0.07,86376.0,-86284.0,50893.0,1764.90,46.0,L-SC,4.55,268.0,-1.00,0.36,26.57,OX40N,NTT,JEWELLERY
45,JCHAC,2282.00,-4.24,-37.58,60.20,-0.01,48023.0,-48033.0,79772.0,16226.09,35.0,M-SC,13.03,233.0,-1.00,0.56,2.10,OX40N,NTT,AC
30,HAPPSTMNDS,1480.71,-3.93,-43.30,226.04,84.85,164250.0,-55501.0,72664.0,-32.68,28.0,H-SC,13.70,147.0,-0.34,0.51,0.00,AR,ATH,IT
28,GLAXO,3466.20,-3.76,-4.29,45.38,39.15,86556.0,-8548.0,190736.0,-23.10,28.0,X-MC,10.68,60.0,-0.10,1.35,23.47,X40,ATH,PHARMA
13,BERGEPAINT,680.00,-3.50,-6.37,29.39,21.14,62556.0,-14491.0,212848.0,-13.02,36.0,X-MC,1.43,74.0,-0.23,1.50,18.20,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-1.40,-7.31,124.32,107.91,170128.0,-10797.0,136847.0,-24.60,37.0,M-SC,11.63,216.0,-0.06,0.97,2.04,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,-0.39,-20.26,25.43,0.02,35396.0,-35369.0,139190.0,112.05,33.0,M-SC,8.98,234.0,-1.00,0.98,21.07,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,-1.08,-20.39,25.72,0.09,20635.0,-20546.0,80231.0,86.13,34.0,M-SC,3.05,253.0,-1.00,0.57,5.41,OX40N,NTT,DURABLES
48,KANSAINER,340.00,-0.26,-21.95,45.39,13.47,95534.0,-59193.0,210474.0,-67.92,53.0,H-SC,6.79,158.0,-0.62,1.48,7.02,XY24,NTT,PAINTS
68,SIS,528.00,-0.77,-23.38,58.27,21.26,49569.0,-25964.0,85068.0,2007.77,52.0,H-SC,4.60,166.0,-0.52,0.60,14.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-0.22,8.14,59.3,72.27,108660.0,13789.0,183237.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.29,14.82,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.22,8.14,59.30,72.27,108660.0,13789.0,183237.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.29,14.82,XR,NTT,IT
6,ASIANTILES,137.00,-0.19,-1.07,83.65,81.67,77629.0,-1008.0,92802.0,8355.56,66.0,L-SC,15.91,271.0,-0.01,0.65,79.72,XR,NTT,CERAMICS
39,INDIAMART,4810.62,2.16,-3.45,114.10,106.72,135877.0,-4250.0,119086.0,-52.15,45.0,H-SC,5.31,139.0,-0.03,0.84,19.13,AR,ATH,MISC
26,FINCABLES,1641.55,-1.40,-7.31,124.32,107.91,170128.0,-10797.0,136847.0,-24.60,37.0,M-SC,11.63,216.0,-0.06,0.97,2.04,OX40N,ATH,CABLES
49,KPIGREEN,730.66,17.13,-4.63,54.08,46.95,64625.0,-5799.0,119498.0,-11.74,63.0,H-SC,9.73,154.0,-0.09,0.84,49.70,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,18931.72,-0.76,-21.64,60.89,26.07,93144.0,-42250.0,152971.0,-61.93,24.0,X-MC,15.49,56.0,-0.45,1.08,0.00,X40N,ATH,IT
9,BAJAJHFL,181.50,-0.00,-19.33,92.74,55.47,182002.0,-47036.0,196250.0,-28.46,26.0,X-MC,16.99,64.0,-0.26,1.38,0.56,X40N,ATH,FINANCE
28,GLAXO,3466.20,-3.76,-4.29,45.38,39.15,86556.0,-8548.0,190736.0,-23.10,28.0,X-MC,10.68,60.0,-0.10,1.35,23.47,X40,ATH,PHARMA
35,HONAUT,58357.33,0.02,-18.97,76.87,43.32,101453.0,-30892.0,131980.0,-30.40,29.0,X-SC,6.92,90.0,-0.30,0.93,1.34,X40N,ATH,ELECTRICAL
54,PAGEIND,51605.08,-2.04,-11.71,45.10,28.11,80199.0,-23585.0,177825.0,-35.98,30.0,X-MC,15.03,55.0,-0.29,1.25,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,680.0,-3.50,-6.37,29.39,21.14,62556.0,-14491.0,212848.0,-13.02,36.0,X-MC,1.43,74.0,-0.23,1.50,18.20,XY24,NTT,PAINTS
67,SIEMENS,4671.5,-0.81,-18.60,54.20,25.51,82099.0,-34620.0,151475.0,-7.09,36.0,H-LC,1.45,49.0,-0.42,1.07,11.63,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,-0.41,4.59,15.47,20.77,37885.0,10750.0,244894.0,-4.44,49.0,X-MC,1.65,62.0,0.28,1.73,11.83,X40N,NTT,BREWERIES
21,DABUR,735.0,1.30,-3.10,48.29,43.70,116084.0,-7678.0,240390.0,-11.07,44.0,X-MC,1.98,72.0,-0.07,1.70,11.49,XY24,BTT,FMCG
44,ITC,452.0,-0.50,-1.82,12.83,10.78,30273.0,-4365.0,235953.0,-41.56,43.0,X-LC,2.02,5.0,-0.14,1.66,3.32,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-0.30,-39.49,123.44,35.20,96108.0,-50812.0,77858.0,2.09,32.0,X-SC,17.34,92.0,-0.53,0.55,0.0,X40,NTT,FOOTWEAR
20,COLPAL,3726.84,-0.81,-22.03,81.50,41.51,167352.0,-58025.0,205340.0,-7.19,30.0,X-MC,8.40,61.0,-0.35,1.45,0.0,XY25,ATH,FMCG
23,DIXON,18931.72,-0.76,-21.64,60.89,26.07,93144.0,-42250.0,152971.0,-61.93,24.0,X-MC,15.49,56.0,-0.45,1.08,0.0,X40N,ATH,IT
54,PAGEIND,51605.08,-2.04,-11.71,45.10,28.11,80199.0,-23585.0,177825.0,-35.98,30.0,X-MC,15.03,55.0,-0.29,1.25,0.0,X40,ATH,APPARELS
3,ACC,3906.00,0.40,-27.21,125.51,64.15,217396.0,-64741.0,173210.0,-57.62,33.0,X-SC,5.37,82.0,-0.30,1.22,0.4,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.0,-0.05,-29.15,101.63,42.84,46805.0,-18952.0,46054.0,-53.72,46.0,X-SC,32.05,83.0,-0.40,0.32,4.19,XY24,NTT,MISC
60,RELAXO,1176.0,-0.72,-48.14,193.71,52.31,145815.0,-69885.0,75275.0,-44.83,40.0,X-SC,7.36,91.0,-0.48,0.53,0.76,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.0,-0.30,-39.49,123.44,35.20,96108.0,-50812.0,77858.0,2.09,32.0,X-SC,17.34,92.0,-0.53,0.55,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.0,-0.19,-3.87,29.07,24.08,35476.0,-4912.0,122038.0,-11.59,38.0,X-SC,7.49,89.0,-0.14,0.86,14.99,XY24,NTT,HEALTHCARE
61,RELIANCE,1952.0,-0.38,-0.67,26.77,25.92,35037.0,-879.0,130883.0,-4.24,49.0,X-LC,9.22,17.0,-0.03,0.92,33.05,XY25,BTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-0.14,-10.73,35.21,20.71,118893.0,-40570.0,337667.0,-22.98,54.0,X-LC,3.10,1.0,-0.34,2.38,12.83,X40,ATH,IT
42,INFY,1972.00,-1.40,10.39,21.61,34.24,74641.0,32513.0,345401.0,-14.21,54.0,X-LC,7.26,2.0,0.44,2.44,19.53,X40,NTT,IT
77,TMPV,600.00,0.88,-25.86,65.86,22.98,110309.0,-58408.0,167490.0,-23.08,53.0,X-LC,6.15,3.0,-0.53,1.18,5.34,XY24,NTT,AUTO
83,VBL,671.64,0.36,-2.14,38.64,35.67,119428.0,-6763.0,309079.0,-13.74,58.0,X-LC,3.03,4.0,-0.06,2.18,11.42,X40N,ATH,FMCG
44,ITC,452.00,-0.50,-1.82,12.83,10.78,30273.0,-4365.0,235953.0,-41.56,43.0,X-LC,2.02,5.0,-0.14,1.66,3.32,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,0.23,-37.87,113.90,32.89,54633.0,-29240.0,47966.0,-692.31,50.0,L-MC,6.71,259.0,-0.54,0.34,32.09,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.19,-1.07,83.65,81.67,77629.0,-1008.0,92802.0,8355.56,66.0,L-SC,15.91,271.0,-0.01,0.65,79.72,XR,NTT,CERAMICS
51,MASFIN,398.61,0.61,-3.72,26.76,22.05,25244.0,-3645.0,94335.0,-16.89,49.0,H-SC,6.52,164.0,-0.14,0.67,37.12,XR,ATH,FINANCE
14,BSOFT,831.70,-2.97,-23.20,96.34,50.79,104065.0,-32635.0,108018.0,-1.43,50.0,H-SC,11.02,151.0,-0.31,0.76,25.77,XR,ATH,IT
61,RELIANCE,1952.00,-0.38,-0.67,26.77,25.92,35037.0,-879.0,130883.0,-4.24,49.0,X-LC,9.22,17.0,-0.03,0.92,33.05,XY25,BTT,REFINERIES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.19,-1.07,83.65,81.67,77629.0,-1008.0,92802.0,8355.56,66.0,L-SC,15.91,271.0,-0.01,0.65,79.72,XR,NTT,CERAMICS
49,KPIGREEN,730.66,17.13,-4.63,54.08,46.95,64625.0,-5799.0,119498.0,-11.74,63.0,H-SC,9.73,154.0,-0.09,0.84,49.70,MH,ATH,POWER
14,BSOFT,831.70,-2.97,-23.20,96.34,50.79,104065.0,-32635.0,108018.0,-1.43,50.0,H-SC,11.02,151.0,-0.31,0.76,25.77,XR,ATH,IT
76,TITAGARH,1548.00,-3.21,-17.29,76.21,45.74,152647.0,-41879.0,200298.0,10.09,62.0,H-SC,5.33,174.0,-0.27,1.41,30.27,XY24,NTT,ENGINEERING
47,JSWINFRA,342.00,1.07,-2.38,19.27,16.43,41719.0,-5278.0,216496.0,-17.32,61.0,X-MC,7.83,66.0,-0.13,1.53,27.77,X40N,NTT,REALTY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.02
1,25,44.21
2,50,75.15


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.15
MC    30.18
LC    25.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.48
X40      23.53
X40N     12.65
XY25      9.92
XR        9.62
AR        8.64
OX40N     7.35
SR        0.97
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.76
H-SC    23.53
X-LC    22.27
M-SC    11.26
X-SC     7.84
H-MC     4.72
L-SC     1.52
M-MC     1.36
M-LC     1.29
H-LC     1.07
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.12,-7.55,42.13
IT,12.57,-18.00,77.34
FINANCE,10.43,-16.61,66.12
MISC,7.02,-30.39,83.74
ELECTRICAL,5.83,-13.51,55.09
PAINTS,5.67,-14.67,31.70
INSURANCE,4.69,-2.02,36.93
PHARMA,3.97,-4.57,37.49
AUTO,2.81,-31.43,76.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3165166.0,21
AR,1348732.0,10
XR,1297092.0,13
X40,1060113.0,15
X40N,891742.0,9
OX40N,751507.0,10
XY25,400712.0,7
SR,282543.0,2
MH,64625.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3641135.0,25
M-SC,1463185.0,15
X-MC,1459639.0,16
X-LC,907003.0,13
X-SC,805634.0,8
H-MC,410425.0,3
L-SC,245659.0,3
M-LC,108660.0,1
H-LC,82099.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246617.0      6
           AR         919834.0      5
M-SC       XY24       823459.0      6
H-SC       XR         794553.0      7
X-MC       X40        498044.0      7
           XY24       412607.0      3
X-LC       X40        375394.0      6
X-MC       X40N       354750.0      4
H-SC       OX40N      332963.0      4
M-SC       OX40N      332168.0      5
X-SC       X40N       319282.0      3
           XY24       299677.0      3
H-SC       SR         282543.0      2
H-MC       AR         219204.0      2
X-LC       X40N       217710.0      2
X-MC       XY25       194238.0      2
X-LC       XY24       191585.0      2
H-MC       XY24       191221.0      1
X-SC       X40        186675.0      2
M-SC       XR         179963.0      2
L-SC       XR         159283.0      2
M-SC       AR         127595.0      2
X-LC       XY25       122314.0      3
M-LC       XR         108660.0      1
L-SC       OX40N       86376.0      1
H-LC       AR          82099.0      1
H-SC       MH          64625.0      1
L-MC       XR          54633.0      1
M-MC       XY25        52028.0      1
L-LC       XY25        32132.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
